<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/optimization_practice/transportation_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transportation Network

* [Problem Statement][1]
* Learn how to use `Suffix`, constraint using decorator and be called in sensitivity analysis

[1]:https://github.com/jckantor/ND-Pyomo-Cookbook/blob/main/notebooks/03.01-Transportation-Networks.ipynb

In [1]:
import pandas as pd
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.0 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-2_amd64.deb ...
Unpacking coinor-libcgl1 (0.60.3+repack1-2) ...
Selecting previously unselecte

In [2]:
Demand = {
   'Lon': 125,        # London
   'Ber': 175,        # Berlin
   'Maa': 225,        # Maastricht
   'Ams': 250,        # Amsterdam
   'Utr': 225,        # Utrecht
   'Hag': 200         # The Hague
}

Supply = {
   'Arn': 600,        # Arnhem
   'Gou': 650         # Gouda
}

T = {
    ('Lon', 'Arn'): 1000,
    ('Lon', 'Gou'): 2.5,
    ('Ber', 'Arn'): 2.5,
    ('Ber', 'Gou'): 1000,
    ('Maa', 'Arn'): 1.6,
    ('Maa', 'Gou'): 2.0,
    ('Ams', 'Arn'): 1.4,
    ('Ams', 'Gou'): 1.0,
    ('Utr', 'Arn'): 0.8,
    ('Utr', 'Gou'): 1.0,
    ('Hag', 'Arn'): 1.4,
    ('Hag', 'Gou'): 0.8
}

In [24]:
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)
C = Demand.keys()
S = Supply.keys()
model.x = Var(C, S, domain=NonNegativeReals)

cost = sum(sum(model.x[c, s] * T[(c, s)] for s in S) for c in C)
model.cost = Objective(expr=cost, sense=minimize)

@model.Constraint(S)
def src(m, s):
    return sum([model.x[c,s] for c in C]) <= Supply[s]
@model.Constraint(C)
def dmd(m, c):
    return sum([model.x[c,s] for s in S]) == Demand[c]

solver = SolverFactory('cbc')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 1705.0, 'Upper bound': 1705.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 13, 'Number of nonzeros': 6, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 1}}, 'Error rc': 0, 'Time': 0.016892194747924805}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Sensitivity Analysis

* `cost_demand_margin` indicates how much cost will increase if demand will increase 1 ton from the customer
* `cost_supply_margin` indicates how much cost will increase if the supply will increase 1 ton

In [31]:
print(f'Optimized Cost is {model.cost()} euro')

trans_df = pd.DataFrame()
for c in C:
  for s in S:
    trans_df.loc[c, s] = model.x[c, s]()
  trans_df.loc[c, 'demand'] = Demand[c]
  trans_df.loc[c, 'cost_demand_margin'] = model.dual[model.dmd[c]]
trans_df['opt_demand'] = sum(trans_df[s] for s in S)

for s in S:
  trans_df.loc['opt_supply', s] = sum(trans_df.loc[c, s] for c in C)
  trans_df.loc['supply', s] = Supply[s]
  trans_df.loc['cost_supply_margin'] = model.dual[model.src[s]]
trans_df

Optimized Cost is 1705.0 euro


,Arn,Gou,demand,cost_demand_margin,opt_demand
Lon,0.0,125.0,125.0,2.5,125.0
Ber,175.0,0.0,175.0,2.7,175.0
Maa,225.0,0.0,225.0,1.8,225.0
Ams,0.0,250.0,250.0,1.0,250.0
Utr,200.0,25.0,225.0,1.0,225.0
Hag,0.0,200.0,200.0,0.8,200.0
opt_supply,600.0,600.0,NaN,NaN,NaN
supply,600.0,650.0,NaN,NaN,NaN
cost_supply_margin,0.0,0.0,0.0,0.0,0.0
